# Implementação do Algoritmo Simplex 
versão Julia

In [1]:
# Bibliotecas
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Funcao para imprimir tableu
def print_tableau(tableau):
    lins = len(tableau)
    cols = len(tableau[0])
    
    header =  ['x' + str(i+1) for i in range(cols-1)]
    print('           ' + ''.join(f'{col:>8}' for col in header))
    print('              ' + '-' * (7 * cols))
    
    print('F.O. ' + ''.join(f'{val:8.2f}' for val in tableau[0]))
    for i in range(1,lins-1):
        print(f'x{tableau[-1][i-1]+1} = ' + ''.join(f'{val:8.2f}' for val in tableau[i]))
    
    print()

# Imprime somente a matriz A
def print_matrix(matrix):
    for linha in matrix:
        print(''.join(f'{val:8.2f}' for val in linha))

# Fase 1: obter solução básica viável

In [3]:
# Procura alguma coluna da matriz Identidade
def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1

# Retorna o indice onde esta "1"
def get_one(vector):
    for i in range(len(vector)):
        if vector[i] == 1:
            return i

def is_zero(element):
    return abs(element) <= 10e-4



# Funcao que guarda o indice das variaveis basicas
# associado a coluna da matriz identidade
def basicas(A):
    id = [0] * len(A)       # coluna da identidade
    i_B = [-1] * len(A)     # indice da variavel
    colunas = np.array(A).T
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and id[get_one(colunas[coluna])] == 0:
            i_B[get_one(colunas[coluna])]= coluna
            id[get_one(colunas[coluna])] = 1
    return id, i_B
# OBS: quando escreve basicas(A)[0] --> acessa quais colunas
#                     basicas(A)[1] --> acessa os indices (-1) das basicas

def fase1(c, A, b):
    nvar = len(A[0])
    d = (False, nvar)
    # Verifica se é necessario adicionar variaveis artificiais
    tem_sol_basica = any(x == -1 for x in basicas(A)[1])
    if not tem_sol_basica:
        d = (False, nvar)
        return c, A, b, d
 

    # Custos da nova funcao
    c1 = [0] * len(c)    # zero para as variaveis originais
    id = basicas(A)[0]
    for i in range(0, len(id)):
        if id[i] == 0:
            coluna = [0] * len(id)
            coluna[i] = 1
            for linha in range(0, len(A)):
                A[linha].append(coluna[linha])
            c1.append(1)  # um para as variaveis artificiais
    
    # se precisou adicionar variaveis
    d = (True, nvar)
    
    return c1, A, b, d

A fase 1 deve retornar um Tableau !!

Usamos somente as colunas:

    - da funcao objetivo e valores das basicas
    - das variaveis originais
    
OBS: se a solucao encontrada for degenerada entao excluimos a linha de zeros

In [4]:
"""
# Reorganiza o tableau
def resultado_fase1(tableau, nvar):
    # mantem as colunas das variaveis originais
    if abs(tableau[0][0]) >= 10e-4:
        return None, None

    qtdd_colunas = nvar + 1


    for basica in range(1, len(tableau)-1):   # [1,0]
        if tableau[-1][basica] > nvar-1:
            for j in range(1,nvar+1):
                if not is_zero(tableau[basica][j]):
                    tableau = atualiza_tableau(tableau, (basica, j))
                else:
                   tableau.pop(basica)

    novo_A = []
    novo_b = []
    

    # mantem as linhas das variaveis basicas originais
    for i, xB in enumerate(tableau[-1]):
        if xB < qtdd_colunas:
            novo_A.append(tableau[i+1][1:qtdd_colunas])
            novo_b.append(tableau[i+1][0])
    return novo_A, novo_b
"""

'\n# Reorganiza o tableau\ndef resultado_fase1(tableau, nvar):\n    # mantem as colunas das variaveis originais\n    if abs(tableau[0][0]) >= 10e-4:\n        return None, None\n\n    qtdd_colunas = nvar + 1\n\n\n    for basica in range(1, len(tableau)-1):   # [1,0]\n        if tableau[-1][basica] > nvar-1:\n            for j in range(1,nvar+1):\n                if not is_zero(tableau[basica][j]):\n                    tableau = atualiza_tableau(tableau, (basica, j))\n                else:\n                   tableau.pop(basica)\n\n    novo_A = []\n    novo_b = []\n    \n\n    # mantem as linhas das variaveis basicas originais\n    for i, xB in enumerate(tableau[-1]):\n        if xB < qtdd_colunas:\n            novo_A.append(tableau[i+1][1:qtdd_colunas])\n            novo_b.append(tableau[i+1][0])\n    return novo_A, novo_b\n'

In [5]:

# Reorganiza o tableau
def resultado_fase1(tableau, nvar):
    # mantem as colunas das variaveis originais
    if not is_zero(tableau[0][0]):
        return None, None

    qtdd_colunas = nvar + 1

    # Verify tableau has valid dimensions before accessing
    if len(tableau) < 2:
        return None, None

    # Obtem os indices das variaveis basicas
    basic_vars = tableau[-1] if len(tableau[-1]) > 0 else []

    novo_A = []
    novo_b = []

    # Processa cada linha, exceto a primeira e a última
    for basica in range(1, len(tableau)-1):
        if basica < len(tableau) and basica < len(basic_vars) and basic_vars[basica] > nvar-1:
            for j in range(1, nvar+1):
                if j < len(tableau[basica]) and not is_zero(tableau[basica][j]):
                    tableau = atualiza_tableau(tableau, (basica, j))

    # mantem as linhas das variaveis basicas originais
    if len(basic_vars) > 0:
        for i, xB in enumerate(basic_vars):
            if i+1 < len(tableau) and xB < qtdd_colunas:
                row = tableau[i+1]
                if len(row) >= qtdd_colunas:
                    novo_A.append(row[1:qtdd_colunas])
                    novo_b.append(row[0])

    return novo_A, novo_b

# Fase 2: Aplicar Método Simplex

## Tableau Inicial 

Estamos assumindo que para montar o Tableau está sendo passada uma matriz A que contenha todas as colunas da identidade necessarias para obter uma solucao basica viável.

OBS: garantimos isso pois antes todos os problemas passam pela fase 1


In [6]:
# Monta o Tableau Inicial
def tableau_inicial(c, A, b):
    linhas_tableau = []
    
    # guarda as basicas
    identidade, indice_basicas = basicas(A)
    print("Indice basicas: ",indice_basicas)
    print("A: ",A)
    print("b: ",b)
    print("c: ",c)
    lz = []
    
    
    fo = 0
    for i in range(0, len(b)):
        fo += b[i] * c[indice_basicas[i]]
    fo = fo * -1     
    lz.append(fo)


    for i in range(0, len(A[0])):
        if i not in indice_basicas:
            col = [A[j][i] for j in range(0, len(A))] #-0,25 0.75
            cust = [c[i] for i in indice_basicas] # 3,4
            el = sum([a*b for a,b in zip(col,cust)])
            el = c[i] - el
            lz.append(el)
        else:
            lz.append(0)

    print("lz: ",lz)
            

    # funcao objetivo e custos reduzidos
    
    linhas_tableau.append(lz)  # linha 0

    # Concatena linhas de A com linhas de b
    for i in range(len(b)):
        linha = [b[i]] + A[i]
        #print("The linha is: ",linha)
        linhas_tableau.append(linha)

    print("+++++++++++++++")

    # indice das variaveis basicas
    linhas_tableau.append(indice_basicas)  # ultima linha
    return linhas_tableau
        

In [7]:
# Monta o Tableau Inicial
def tableau_inicial_simples(c, A, b):
    linhas_tableau = []
    
    # guarda as basicas
    identidade, indice_basicas = basicas(A)
    
    # funcao objetivo e custos reduzidos
    funcao_obj = (-1) * sum(b)
    custos = []
    for j in range(len(A[0])):
        if j not in indice_basicas:
            c_j = 0
            for i in range(len(A)):
                c_j += A[i][j]
            custos.append((-1)*c_j)
        else: 
            custos.append(0)
    
    linhas_tableau.append([funcao_obj]+custos)  # linha 0

    # Concatena linhas de A com linhas de b
    for i in range(len(b)):
        linha = [b[i]] + A[i]
        #print("The linha is: ",linha)
        linhas_tableau.append(linha)

    print("+++++++++++++++")

    # indice das variaveis basicas
    linhas_tableau.append(indice_basicas)  # ultima linha
    return linhas_tableau

In [8]:
# Verifica se tem algum custo reduzido negativo
def nao_otima(tableau):
    return any(c < 0 for c in tableau[0][1:])

# Define elemento pivo
def posicao_pivo(tableau):
    custos = tableau[0]
    # Escolhe o primeiro custo reduzido negativo
    # "xj entra na base"
    for j in range(1,len(custos)):
        if custos[j] < 0:
            coluna_pivo = j
            break

    theta = []
    for linha_i in range (1,len(tableau)-1):
        if tableau[linha_i][coluna_pivo] <= 0:
            theta.append(math.inf)
        else:
            theta.append(tableau[linha_i][0] / tableau[linha_i][coluna_pivo])
        
    # Escolhe o menor indice das variaveis que podem sair da base
    # soma um porque estamos ignorando a linha de custos
    linha_pivo = theta.index(min(theta)) + 1
    
    return linha_pivo, coluna_pivo

# Atualiza o tableau com base no pivo encontrado
def atualiza_tableau(tableau, pivo):
    i, j = pivo
    valor_pivo = tableau[i][j]
    
    # Divide a linha do pivo para que o valor do pivo seja 1
    tableau[i] = np.array(tableau[i]) / valor_pivo
    
    # Atualiza os valores de dentro do tableau
    # ou seja, zera os outros elementos da coluna
    for linha in range (len(tableau)-1):
        if linha != i:
            multiplicador = (-1) * tableau[linha][j]
            tableau[linha] = multiplicador*tableau[i] + tableau[linha]

            tableau[linha] = np.array([0 if is_zero(x) else x for x in list(tableau[linha])])
                
   
    # Atualiza variaveis básicas
    tableau[-1][i-1] = j-1 

    return tableau

def otima(tableau, nvar):
    # inicia com todos valendo zero

    #print("Final")
    #print(type(tableau))


    solucoes = [0] * nvar
    
    # coloca o valor das basicas na posicao correta da solucao
    for i, xB in enumerate(tableau[-1]):
        solucoes[xB] = tableau[i+1][0]
        
    return solucoes        




In [9]:
# SIMPLEX para resolver FASE 1
def simplex(c, A, b, d, t_fase1=False):
    if t_fase1:
        tableau = tableau_inicial(c, A, b)
    else:
        tableau = tableau_inicial_simples(c, A, b)

    print("Tableau inicial: ")
    print_tableau(tableau)

    nvar = d[1]

    while nao_otima(tableau):
        pivo = posicao_pivo(tableau)
        tableau = atualiza_tableau(tableau, pivo)
        print("Tableau atualizado: ")
        print_tableau(tableau)
    
    if d[0]:
        novo_A, novo_b = resultado_fase1(tableau, nvar)
        return novo_A, novo_b
    else:
        print("Tableau final")
        print_tableau(tableau)
        return A, otima(tableau, nvar)    


# Testando exemplo dos slides aula 23 #
Solucão básica viável para iniciar o simplex:
(x1,x2,x3,x4) = (1 ; 1/2 ; 1/3 ; 0) e zero nas variaveis a mais

Problema: A tem linhas LD --> sol. encontrada na fase 1 é degenerada

In [10]:
# Coeficientes da função objetivo
c = [1, 1, 1, 0]

# Matriz de coeficientes das restrições
A = [
    [1, 2, 3, 0],
    [-1, 2, 6, 0],
    [0, 4, 9, 0],
    [0, 0, 3, 1]
]

# Vetor de termos independentes das restrições
b = [3, 2, 5, 1]

In [11]:
def get_solution_simplex(c, A, b):
    nvar = len(A[0])
    d = (False, nvar)
    c1, A1, b1, d = fase1(c, A, b)

    # Caso seja necessario usar variaveis artificiais(Faze 1)
    if d[0]:
        print("Simplex FASE 1: ")
        A2, b2 = simplex(c1,A1,b1,d)
        if A2 is None:
            print("Problema infactivel")
            return None
        else:
            print("Resultado da fase 1: ",b2)


        # Convertendo tudo para mesmo formato que o simplex(Faze 2)
        A2 = [[float(x) for x in row] for row in A2]
        b2 = [float(x) for x in b2] 
        
        fA, fb = simplex(c, A2, b2,(False, nvar), True)
        print("Resultado da fase 2: ",fb)
    else:  # Não foi necessario usar a fase 1(Resolve direto Faze 2)
        print("Nao Foi nessesario usar a fase 1!!")
        fA, fb = simplex(c, A1, b1, d)
        print("Resultado da fase 2: ", fb)

In [12]:
get_solution_simplex(c, A, b)


Simplex FASE 1: 
+++++++++++++++
Tableau inicial: 
                 x1      x2      x3      x4      x5      x6      x7
              --------------------------------------------------------
F.O.   -11.00    0.00   -8.00  -21.00    0.00    0.00    0.00    0.00
x5 =     3.00    1.00    2.00    3.00    0.00    1.00    0.00    0.00
x6 =     2.00   -1.00    2.00    6.00    0.00    0.00    1.00    0.00
x7 =     5.00    0.00    4.00    9.00    0.00    0.00    0.00    1.00
x4 =     1.00    0.00    0.00    3.00    1.00    0.00    0.00    0.00

Tableau atualizado: 
                 x1      x2      x3      x4      x5      x6      x7
              --------------------------------------------------------
F.O.    -3.00   -4.00    0.00    3.00    0.00    0.00    4.00    0.00
x5 =     1.00    2.00    0.00   -3.00    0.00    1.00   -1.00    0.00
x2 =     1.00   -0.50    1.00    3.00    0.00    0.00    0.50    0.00
x7 =     1.00    2.00    0.00   -3.00    0.00    0.00   -2.00    1.00
x4 =     1.00    0.

In [13]:

# Objective function coefficients (minimization, so we use negative values)
c = [-2, -1, 0, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [-1, 1, 1, 0],  
    [1, 2, 0, 1]   
]

# RHS values
b = [-1, 7]

get_solution_simplex(c, A, b)

Nao Foi nessesario usar a fase 1!!
+++++++++++++++
Tableau inicial: 
                 x1      x2      x3      x4
              -----------------------------------
F.O.    -6.00    0.00   -3.00    0.00    0.00
x3 =    -1.00   -1.00    1.00    1.00    0.00
x4 =     7.00    1.00    2.00    0.00    1.00

Tableau atualizado: 
                 x1      x2      x3      x4
              -----------------------------------
F.O.    -9.00   -3.00    0.00    3.00    0.00
x2 =    -1.00   -1.00    1.00    1.00    0.00
x4 =     9.00    3.00    0.00   -2.00    1.00

Tableau atualizado: 
                 x1      x2      x3      x4
              -----------------------------------
F.O.     0.00    0.00    0.00    1.00    1.00
x2 =     2.00    0.00    1.00    0.33    0.33
x1 =     3.00    1.00    0.00   -0.67    0.33

Tableau final
                 x1      x2      x3      x4
              -----------------------------------
F.O.     0.00    0.00    0.00    1.00    1.00
x2 =     2.00    0.00    1.00    0.3

In [14]:
# Objective function coefficients
c = [4, 3, 7]  # Added zeros for slack variables

# Constraint matrix with slack variables
A = [
    [2, 2, 1],  # Added slack variables [1, 0] for first constraint
    [3, 1, 2]   # Added slack variables [0, 1] for second constraint
]

# RHS values remain the same
b = [12, 8]




get_solution_simplex(c, A, b)

Simplex FASE 1: 
+++++++++++++++
Tableau inicial: 
                 x1      x2      x3      x4      x5
              ------------------------------------------
F.O.   -20.00   -5.00   -3.00   -3.00    0.00    0.00
x4 =    12.00    2.00    2.00    1.00    1.00    0.00
x5 =     8.00    3.00    1.00    2.00    0.00    1.00

Tableau atualizado: 
                 x1      x2      x3      x4      x5
              ------------------------------------------
F.O.    -6.67    0.00   -1.33    0.33    0.00    1.67
x4 =     6.67    0.00    1.33   -0.33    1.00   -0.67
x1 =     2.67    1.00    0.33    0.67    0.00    0.33

Tableau atualizado: 
                 x1      x2      x3      x4      x5
              ------------------------------------------
F.O.     0.00    0.00    0.00    0.00    1.00    1.00
x2 =     5.00    0.00    1.00   -0.25    0.75   -0.50
x1 =     1.00    1.00    0.00    0.75   -0.25    0.50

Resultado da fase 1:  [np.float64(5.0), np.float64(1.0)]
Indice basicas:  [1, 0]
A:  [[0.0,

In [16]:

# Objective function coefficients
c = [2, -1, 5, 0]  # Added zeros for slack variables

# Constraint matrix with slack variables
A = [
    [1, 1, 1, 0], 
    [2, 3, 2, 1] 
]

# RHS values remain the same
b = [4, 5]




get_solution_simplex(c, A, b)

Simplex FASE 1: 
+++++++++++++++
Tableau inicial: 
                 x1      x2      x3      x4      x5
              ------------------------------------------
F.O.    -9.00   -3.00   -4.00   -3.00    0.00    0.00
x5 =     4.00    1.00    1.00    1.00    0.00    1.00
x4 =     5.00    2.00    3.00    2.00    1.00    0.00

Tableau atualizado: 
                 x1      x2      x3      x4      x5
              ------------------------------------------
F.O.    -1.50    0.00    0.50    0.00    1.50    0.00
x5 =     1.50    0.00   -0.50    0.00   -0.50    1.00
x1 =     2.50    1.00    1.50    1.00    0.50    0.00

Problema infactivel
